In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-142.373734,271.956177,-16.337118,51.671291,76.822624,-245.059525,27.996546,38.375412,261.649292,-244.260834,...,512.339355,-60.587494,-10.584517,370.291687,-274.114868,-321.223785,312.296051,1,8077,critical
1,-2.426456,5.172460,0.183453,1.040403,0.761608,-3.009145,-0.883193,0.836070,2.778491,-2.877727,...,10.506008,-3.874928,1.081834,7.217535,-4.412273,-6.149477,5.995068,1,5553,critical
2,-5.216374,12.301514,-0.116449,2.123092,3.188100,-7.073945,-0.892327,1.628437,7.502768,-8.853976,...,19.812071,-4.468370,1.009261,14.174513,-9.546618,-12.650854,13.139441,1,9341,critical
3,-10.633791,23.823032,-1.599639,3.053346,3.419122,-15.894526,-2.235992,3.425864,15.858608,-15.242412,...,30.434586,-7.688050,1.636729,25.253891,-21.265335,-20.495380,22.952433,1,10419,critical
4,-142.373734,271.956177,-16.337118,51.671291,76.822624,-245.059525,27.996546,38.375412,261.649292,-244.260834,...,512.339355,-60.587494,-10.584517,370.291687,-274.114868,-321.223785,312.296051,1,12232,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8232
F1_macro: 0.7666
Accuracy: 0.8232
Precision_macro: 0.7757
Recall_macro: 0.7595

Bagging Metrics:
F1_micro: 0.8302
F1_macro: 0.7701
Accuracy: 0.8302
Precision_macro: 0.7889
Recall_macro: 0.7573

XGBoost Metrics:
F1_micro: 0.8302
F1_macro: 0.7766
Accuracy: 0.8302
Precision_macro: 0.7842
Recall_macro: 0.7703


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")


Final Test Results:

Random Forest Metrics:
F1_micro: 0.8233
F1_macro: 0.7702
Precision_macro: 0.7747
Recall_macro: 0.7661
Accuracy: 0.8233
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       814
           1       0.68      0.64      0.66       295

    accuracy                           0.82      1109
   macro avg       0.77      0.77      0.77      1109
weighted avg       0.82      0.82      0.82      1109


Bagging Metrics:
F1_micro: 0.8224
F1_macro: 0.7661
Precision_macro: 0.7746
Recall_macro: 0.7590
Accuracy: 0.8224
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       814
           1       0.68      0.62      0.65       295

    accuracy                           0.82      1109
   macro avg       0.77      0.76      0.77      1109
weighted avg       0.82      0.82      0.82      1109


XGBoost Metrics:
F1_micro: 0.8151
F1_macro: 

In [7]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/codebert/result_codebert_base_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")


Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/codebert/result_codebert_base_sum.pkl
